In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import utilities
import GCC_Preprocess as gpp
import pyHREBSD

%matplotlib widget

### Reading in data

Data can be imported using the `utilities.get_scan_data` function. Note that this requires knowledge of the detector used during data collection. The detector pixel size (unbinned) and the detector pixel dimensions (unbinned) permit transforming the pattern center from (xstar, ystar, zstar) into (xpc, ypc, L).

Note that this function does not actually read any patterns. To read in patterns, the function `utilities.get_patterns` must be called. This function also permits selection of a small region of interest within the scan by passing indices of the patterns of interest.

In [ ]:
# Load the scan data and print out some information
up2 = "E:/cells/CoNi90-OrthoCells.up2"
ang = "E:/cells/CoNi90-OrthoCells.ang"
pixel_size = 13.0
Nxy = (2048, 2048)

pat_obj, ang_data = utilities.get_scan_data(up2, ang, Nxy, pixel_size)
print("Scan data loaded")
print("File size:", pat_obj.filesize)
print("Pattern shape:", pat_obj.patshape)
print("Number of patterns:", pat_obj.nPatterns)
print("Scan shape:", ang_data.shape)
print("Ang fields:", ang_data.fields)

# Grab a single pattern and display it
pattern = utilities.get_patterns(pat_obj, [0])
utilities.view(pattern, cmap='gray', title="1st pattern")

In [ ]:
# Grab a subset of the patterns
point = (112, 96)
size = 100

idx = utilities.get_index(point, size, ang_data.shape)
pats = utilities.get_patterns(pat_obj, idx)

### Pattern processing

There are two recommended pattern processing routines.

- **Applying a bandpass filter**: This is done using the difference of gaussians approach. This requires choosing a lower sigma that removes noise and a higher sigma that removes background intensity gradients.
- **Apply adaptive histogram equalization**: This enhances the local contrast in the image to highlight bands.

The `utilities.test_bandpass(pattern)` function will generate four composite images that allow you to tune the sigma values of the bandpass filter. It creates a grid of images with differen sigma values, the same with the histogram equalization applied, and the cross-correlation result of the images both with and without the histogram equalization.

Once an optimal set of sigma values have been identified, the `utilities.process_patterns` function can be called.

Similarly, the sharpness of the patterns can be caluclated using the `utilities.get_sharpness` function.

In [ ]:
# Run the bandpass test to determine what sigma values to use for the HREBSD analysis
utilities.test_bandpass(pats[0, 0])

In [ ]:
# Process the patterns using the bandpass filter
DoG_sigmas = (1.0, 10.0)
pats_processed = utilities.clean_patterns(pats, equalize=True, dog_sigmas=DoG_sigmas)
sharpness = utilities.get_sharpness(pats_processed)

utilities.view(pats[0, :3])
utilities.view(sharpness)

### Running HREBSD

With the patterns processed, we can now run HREBSD. The first step is to define the reference pattern of the image and the subset size to use.

With the reference and subset, we first calculate the initial guesses of the homographies in order to speed up the actual HREBSD analysis. This is done with `gpp.GCC_Initial_Guess`. Note that this function takes time.

Once the initial guesses have been made, we can run HREBSD to get the actual homographies. Homographies are then used to determine the deviatoric deformation gradient tensor and the strain/rotation tensors.

In [ ]:
# Set the reference pattern and the subset slice to use for the HREBSD analysis
R = pats[112, 96]
subset_slice = (slice(R.shape[0] // 2 - 64, R.shape[0] // 2 + 64),
                slice(R.shape[1] // 2 - 64, R.shape[1] // 2 + 64))

# Also set a save_name for the results
save_name = "CoNi90-OrthoCells_R112-96"

In [ ]:
# Calculate the initial guesses for the HREBSD analysis
p0 = gpp.GCC_Initial_Guess(R, pats)
np.save(f"{save_name}_p0.npy", p0)
# p0 = np.load(f"{save_name}_p0.npy", p0)

In [ ]:
# Run the HREBSD analysis using the initial guesses
p = pyHREBSD.get_homography(R, pats, subset_slice=subset_slice, p0=p0, max_iter=50, conv_tol=1e-7)
np.save(f"{save_name}_p.npy", p)
# p = np.load(f"{save_name}_p.npy")

In [ ]:
# Calculate the deviatoric deformation gradient tensor
Fe = pyHREBSD.homography_to_elastic_deformation(p, ang_data.pc)

# Calculate the strain tensor and the rotation tensor
e, w = pyHREBSD.deformation_to_strain(Fe)

In [ ]:
# Save out the results
xy = (50, 50) # The location of the reference point

plt.close('all')
utilities.view_tensor_images(Fe, tensor_type="deformation", xy=xy, save_name=save_name, save_dir="results/")
utilities.view_tensor_images(e, tensor_type="strain", xy=xy, save_name=save_name, save_dir="results/")
utilities.view_tensor_images(w, tensor_type="rotation", xy=xy, save_name=save_name, save_dir="results/")